# Script pour rejouer un certain modèle de DQN
Source: https://medium.com/@www.seymour/training-an-ai-to-play-a-game-using-deep-reinforcement-learning-b63534cfdecd

Dans ce script, il n'y a qu'à changer le numéro et chemin du modèle pour ensuite voir le rendu de l'agent

### Imports

In [9]:
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import dqn, game, record

import importlib
importlib.reload(game)
importlib.reload(dqn)

import pygame

### Modèle à charger

In [13]:
model = 299
folder = os.path.join("..", "simulations","2024-04-21 0227", "models")
record_video = False

### Chargement des éléments de jeu

In [11]:
env = game.Environment()

agent = dqn.DQN(
    state_shape=env.ENVIRONMENT_SHAPE,
    action_size=env.ACTION_SPACE_SIZE
)
agent.load(os.path.join(folder, f"{model}.keras"))

### Script de replay

In [12]:
state = env.reset()
state = np.expand_dims(state, axis=0)

# Initialisation de pygame pour avoir le jeu
pygame.init()
screen = pygame.display.set_mode((env.WINDOW_WIDTH, env.WINDOW_HEIGHT))
clock = pygame.time.Clock()
running = True
score = 0

recorder = None
if record_video:
    recorder = record.ScreenRecorder(env.WINDOW_WIDTH, env.WINDOW_HEIGHT, env.FPS, f"{folder}_{model}.avi")

# Boucle pour jouer au jeu
while running:
    pygame.display.set_caption(f"Score: {score}")

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = agent.act(state, 0)
    state, reward, done, score = env.step(action)
    state = np.expand_dims(state, axis=0)

    env.render(screen)
    pygame.display.flip()
    clock.tick(15)

    if record_video:
        recorder.capture_frame(screen)

pygame.quit()
if record_video:
    recorder.end_recording()